In [11]:
import pickle
from pathlib import Path

import os
# 작업 디렉토리를 crop_pkl로 변경
ROOT = r"D:\golfDataset\dataset\crop_pkl"
os.chdir(ROOT)

def load_all_annotations(pkl_path):
    with open(pkl_path, 'rb') as f:
        data = pickle.load(f)
    return data['annotations']

def make_mmaction_pkl(train_pkl_list, valid_pkl_list, test_pkl_list, out_train, out_test):
    # 1. train/valid
    train_annos, train_dirs, valid_dirs = [], [], []
    for pkl in train_pkl_list:
        annos = load_all_annotations(pkl)
        train_annos.extend(annos)
        train_dirs.extend([anno['frame_dir'] for anno in annos])
    for pkl in valid_pkl_list:
        annos = load_all_annotations(pkl)
        train_annos.extend(annos)
        valid_dirs.extend([anno['frame_dir'] for anno in annos])
    split = {
        'xsub_train': train_dirs,
        'xsub_val': valid_dirs
    }
    train_data = {
        'split': split,
        'annotations': train_annos
    }
    with open(out_train, 'wb') as f:
        pickle.dump(train_data, f)
    print(f"[INFO] Wrote train.pkl ({len(train_annos)} samples)")

    # 2. test
    test_annos, test_dirs = [], []
    for pkl in test_pkl_list:
        annos = load_all_annotations(pkl)
        test_annos.extend(annos)
        test_dirs.extend([anno['frame_dir'] for anno in annos])
    split = {
        'xsub_test': test_dirs
    }
    test_data = {
        'split': split,
        'annotations': test_annos
    }
    with open(out_test, 'wb') as f:
        pickle.dump(test_data, f)
    print(f"[INFO] Wrote test.pkl ({len(test_annos)} samples)")

# 사용 예시
if __name__ == '__main__':
    # 실제 파일명에 맞게 경로를 지정하세요
    train_pkl_list = ['skeleton_dataset_train.pkl']
    valid_pkl_list = ['skeleton_dataset_valid.pkl']
    test_pkl_list  = ['skeleton_dataset_test.pkl']

    make_mmaction_pkl(
        train_pkl_list, valid_pkl_list, test_pkl_list,
        out_train='train.pkl',
        out_test='test.pkl'
    )

[INFO] Wrote train.pkl (1798 samples)
[INFO] Wrote test.pkl (272 samples)


In [ ]:
import pickle
from pathlib import Path

import os
# 작업 디렉토리를 crop_pkl로 변경
ROOT = r"D:\golfDataset\dataset\crop_pkl"
os.chdir(ROOT)

def check_pkl_annotations(pkl_files):
    if isinstance(pkl_files, (str, Path)):
        pkl_files = [pkl_files]
    for pkl_path in pkl_files:
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
        print(f"{Path(pkl_path).name}: annotations 개수 = {len(data['annotations'])}")

# 사용 예시 (파일명만 전달해도 동작)
check_pkl_annotations(['skeleton_dataset_train.pkl'])
check_pkl_annotations(['skeleton_dataset_valid.pkl'])

skeleton_dataset_train.pkl: annotations 개수 = 1618
skeleton_dataset_valid.pkl: annotations 개수 = 180


In [16]:
import pickle
import numpy as np

def check_mmaction2_pkl(pkl_path):
    with open(pkl_path, 'rb') as f:
        data = pickle.load(f)

    # 1. 최상위 키 체크
    print(f"최상위 키: {list(data.keys())}")
    assert 'annotations' in data, "annotations 키가 없습니다."
    assert 'split' in data, "split 키가 없습니다."

    # 2. split 구조 체크
    print(f"split: {data['split']}")
    assert isinstance(data['split'], dict), "split은 dict여야 합니다."
    for k, v in data['split'].items():
        assert isinstance(v, list), f"split[{k}]은 list여야 합니다."

    # 3. annotations 구조 및 필드 체크
    annos = data['annotations']
    print(f"샘플 개수: {len(annos)}")
    required_keys = {'frame_dir', 'label', 'img_shape', 'original_shape', 'total_frames', 'keypoint', 'keypoint_score'}
    for i, a in enumerate(annos):
        keys = set(a.keys())
        if not required_keys.issubset(keys):
            print(f"[{i}] 필드 누락: {required_keys - keys}")
        # label 타입
        if not isinstance(a['label'], int):
            print(f"[{i}] label 타입 오류: {type(a['label'])}")
        # keypoint shape
        kp = np.array(a['keypoint'])
        if kp.ndim != 4 or kp.shape[2] != 17 or kp.shape[3] != 2:
            print(f"[{i}] keypoint shape 오류: {kp.shape}")
        # 프레임 개수
        if kp.shape[1] == 0:
            print(f"[{i}] 프레임 없음: {a['frame_dir']}")
        # NaN/Inf 체크
        if np.isnan(kp).any() or np.isinf(kp).any():
            print(f"[{i}] NaN/Inf 포함: {a['frame_dir']}")
        # 0-only 체크
        if (kp == 0).all():
            print(f"[{i}] 0-only keypoint: {a['frame_dir']}")

    print("✅ MMACTION2 예제 포맷 체크 완료")

# 사용 예시
if __name__ == '__main__':
    check_mmaction2_pkl('D:\\golfDataset\\dataset\\crop_pkl\\train.pkl')

최상위 키: ['split', 'annotations']
split: {'xsub_train': ['20201120_General_027_DOC_A_M40_MM_071_crop', '20201118_General_016_DOC_A_F30_SM_051_crop', '20201127_General_052_DOS_A_F30_MM_025_crop', '20201120_General_028_DOC_A_M40_MM_047_crop', '20201130_General_057_DOS_A_M50_MM_087_crop', '20201116_General_003_DOS_A_F30_MM_048_crop', '20201118_General_016_DOC_A_F30_SM_009_crop', '20201124_General_036_DOS_A_M50_MM_022_crop', '20201130_General_058_DOS_A_M30_MM_003_crop', '20201126_General_048_DOS_A_M30_MM_034_crop', '20201118_General_016_DOC_A_F30_SM_036_crop', '20201118_General_020_DOC_A_F50_MM_062_crop', '20201118_General_013_DOC_A_M30_BS_031_crop', '20201120_General_023_DOC_A_M40_MM_035_crop', '20201124_General_034_DOS_A_M40_MM_064_crop', '20201117_General_006_DOC_A_M40_BS_014_crop', '20201117_General_005_DOC_A_M40_MM_061_crop', '20201116_General_003_DOS_A_F30_MM_013_crop', '20201120_General_027_DOC_A_M40_MM_001_crop', '20201116_General_003_DOS_A_F30_MM_093_crop', '20201127_General_054_DOC

In [17]:
import pickle
with open(r"D:\golfDataset\dataset\crop_pkl\train.pkl", "rb") as f:
    data = pickle.load(f)
print(data['annotations'][0])  # 첫 샘플 출력

{'total_frames': 217, 'img_shape': (1080, 1920), 'original_shape': (1080, 1920), 'keypoint': array([[[[-2.1372440e-05, -1.1398729e-03],
         [ 2.1711434e-05, -1.2422111e-03],
         [ 0.0000000e+00,  0.0000000e+00],
         ...,
         [-2.0872426e-04,  8.9631596e-04],
         [ 2.5222130e-04,  1.7410485e-03],
         [-3.1663524e-04,  1.7543446e-03]],

        [[-2.1336460e-05, -1.1403459e-03],
         [ 2.1770818e-05, -1.2426706e-03],
         [-5.7446356e-05, -1.2165573e-03],
         ...,
         [-2.0872970e-04,  8.9612661e-04],
         [ 2.5221836e-04,  1.7408272e-03],
         [-3.1664092e-04,  1.7541524e-03]],

        [[-2.1394902e-05, -1.1407410e-03],
         [ 2.1760672e-05, -1.2431897e-03],
         [-5.7510730e-05, -1.2045512e-03],
         ...,
         [-2.0888656e-04,  8.9692260e-04],
         [ 2.5241851e-04,  1.7423334e-03],
         [-3.1687887e-04,  1.7556589e-03]],

        ...,

        [[ 0.0000000e+00,  0.0000000e+00],
         [ 0.0000000e+00,  0